# K-fold practice
## 1. Lib. dataload


## 2. PreProcessing
- Noise 처리, Scaling, Labeling, One-hot Encoding ... 

## 3. EDA ==> insight(생략)
- 예, 가설 : 돈이 세상에서 제일 좋다 --> 선실등급이 1등급인 경우, 생존확률이 높다


## 4. Model 생성
- 어떤 고객은 생존했는가? 사망했는가? 
- model : decistion tree, logistic regression, random forest, xgbclassifier

# 5. 모델검증
- classification report를 출력